<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load</a></span><ul class="toc-item"><li><span><a href="#Brain-Region" data-toc-modified-id="Brain-Region-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Brain Region</a></span></li><li><span><a href="#Input-metadata" data-toc-modified-id="Input-metadata-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Input metadata</a></span></li><li><span><a href="#Level-1-Annotation" data-toc-modified-id="Level-1-Annotation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Level 1 Annotation</a></span></li><li><span><a href="#Level-2-Annotation" data-toc-modified-id="Level-2-Annotation-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Level 2 Annotation</a></span></li><li><span><a href="#Level-3-Annotation" data-toc-modified-id="Level-3-Annotation-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Level 3 Annotation</a></span></li></ul></li><li><span><a href="#Combine" data-toc-modified-id="Combine-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Combine</a></span></li><li><span><a href="#Re-define-cell-class-into-three" data-toc-modified-id="Re-define-cell-class-into-three-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Re-define cell class into three</a></span></li><li><span><a href="#Tri-class-UMAP-TSNE" data-toc-modified-id="Tri-class-UMAP-TSNE-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Tri-class UMAP TSNE</a></span></li><li><span><a href="#Update-region-info" data-toc-modified-id="Update-region-info-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Update region info</a></span></li><li><span><a href="#Save-file" data-toc-modified-id="Save-file-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Save file</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Plot</a></span></li></ul></div>

In [1]:
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from cemba_data.plot import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_msgpack('../../../metadata/CellMeta.122687.Raw.msg')

In [3]:
(df['FinalReads'] > 500000).sum()

110294

## Load

### Brain Region

In [4]:
brain_region = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/BrainRegion.txt',
                           sep='\t', index_col='DissectionRegion')
major_region_palette = brain_region.set_index('MajorRegion')['MajorRegionColor'].to_dict()
sub_region_palette = brain_region.set_index('SubRegion')['SubRegionColor'].to_dict()
region_palette = brain_region['DissectionRegionColor'].to_dict()

### Input metadata

In [5]:
input_cell_meta = pd.read_msgpack('/home/hanliu/project/mouse_rostral_brain/metadata/CellMeta.122687.Raw.msg')

In [6]:
input_cell_meta.head()

,AllcPath,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,Region,...,BamFilteringRate,MappingRate,Pos96,Plate,Col96,Row96,Col384,Row384,FACS_Date,Slice
cell,,,,,,,,,,,,,,,,,,,,,
10E_M_0,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.008198,0.822633,0.821166,0.041640,0.033718,1626504.0,4407752,2892347.0,10E,...,0.562347,0.656195,A1,CEMBA190625-10E-1,0,0,0,0,190625,10
10E_M_1,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.006019,0.743035,0.741479,0.024127,0.018218,2009998.0,5524084,3657352.0,10E,...,0.549577,0.662074,A1,CEMBA190625-10E-1,0,0,0,1,190625,10
10E_M_10,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.006569,0.750172,0.748520,0.027665,0.021235,1383636.0,3455260,2172987.0,10E,...,0.636744,0.628892,A10,CEMBA190625-10E-1,9,0,19,0,190625,10
10E_M_100,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.005984,0.750825,0.749325,0.024424,0.018551,377115.0,1088368,703698.0,10E,...,0.535905,0.646563,B10,CEMBA190625-10E-1,9,1,18,2,190625,10
10E_M_1000,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.000000,0.666667,0.666667,0.012780,0.012780,26.0,115216,58.0,10E,...,0.448276,0.000503,C5,CEMBA190625-10E-4,4,2,9,5,190625,10


In [7]:
print('Input Cells:', input_cell_meta.shape[0])

Input Cells: 122687


### Level 1 Annotation

In [8]:
level_1_final = pd.read_msgpack('/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/cell_tidy_data.final_annotation.with_region.msg')

In [9]:
level_1_final = level_1_final[['CellClass', 'umap_0', 'umap_1', 'tsne_0', 'tsne_1']]
# add l1- prefix to coords to distinguish
level_1_final.columns = ['CellClass', 'l1-umap_0', 'l1-umap_1', 'l1-tsne_0', 'l1-tsne_1']
level_1_final.head()

,CellClass,l1-umap_0,l1-umap_1,l1-tsne_0,l1-tsne_1
index,,,,,
10E_M_0,Inh,8.002170,8.983358,4.595703,14.504827
10E_M_1,Exc-HPF,-33.583858,-8.758917,19.721622,22.792822
10E_M_10,Exc-HPF,-33.994816,-8.389750,21.309082,22.672712
10E_M_101,Exc-HPF,-33.219925,-7.649719,22.575315,19.733137
10E_M_102,Exc-HPF,-30.274584,-15.904350,22.628148,0.258059


In [10]:
level_1_final = level_1_final.reindex(input_cell_meta.index)

In [11]:
level_1_final['CellClass'].value_counts()

Exc        53395
Inh        26893
GC          8224
NonN        7920
Exc-HPF     7908
Name: CellClass, dtype: int64

In [12]:
print('Level 1 Non Outlier Cells:', level_1_final['CellClass'].value_counts().sum())

Level 1 Non Outlier Cells: 104340


### Level 2 Annotation

In [13]:
level2_final_paths = list(pathlib.Path('/home/hanliu/project/mouse_rostral_brain/').glob('study/Level2*/*/cell_tidy_data.final_annotation.msg'))
level_2_final = pd.concat([pd.read_msgpack(i) for i in level2_final_paths], sort=True)

In [14]:
level_2_final = level_2_final[['consensus_cluster_rescued_anno', 'umap_0', 'umap_1', 'tsne_0', 'tsne_1']]
level_2_final.columns = ['MajorType', 'l2-umap_0', 'l2-umap_1', 'l2-tsne_0', 'l2-tsne_1']
level_2_final.head()

,MajorType,l2-umap_0,l2-umap_1,l2-tsne_0,l2-tsne_1
10E_M_1278,IT-L5,8.961619,-10.417772,19.368273,40.382910
10E_M_135,NP-L6,25.833487,3.810488,-8.360947,-23.706875
10E_M_1507,CT-L6,24.717123,11.881371,-24.749526,-45.465047
10E_M_1614,NP-L6,25.410795,3.615465,-7.734582,-23.002058
10E_M_1737,PT-L5,8.643658,7.802010,44.712395,12.111033


In [15]:
level_2_final['MajorType'].value_counts()

IT-L23       11919
OLF-Exc       9681
CT-L6         8716
IT-L5         6640
DG            6179
IT-L6         4947
PAL-Inh       4481
IT-L4         4347
CA1           4075
MSN-D2        3340
MSN-D1        3139
ODC           3082
MGE-Sst       2931
PT-L5         2638
MGE-Pvalb     2636
CA3           2423
ASC           2215
LSX-Inh       2143
Foxp2         1964
D1L-Fstl4     1962
NP-L6         1819
CGE-Vip       1746
OLF           1452
CGE-Lamp5     1449
L6b           1069
CLA            945
MGC            853
OPC            777
IG-CA2         745
EP             666
D1L-PAL        495
VLMC           419
CA3-St18       408
Chd7           386
Outlier        358
ANP            331
VLMC-Pia       225
Unc5c          219
PC             170
DG-po          145
Gfra1          110
EC              95
Name: MajorType, dtype: int64

In [16]:
level_2_final = level_2_final.reindex(input_cell_meta.index)

In [17]:
print('Level 2 Non Outlier Cells:', (level_2_final[~level_2_final['MajorType'].isna()]['MajorType'] != 'Outlier').sum())

Level 2 Non Outlier Cells: 103982


### Level 3 Annotation

In [18]:
level3_final_paths = list(pathlib.Path('/home/hanliu/project/mouse_rostral_brain/').glob('study/Level3*/*/cell_tidy_data.final_annotation.msg'))
level_3_final = pd.concat([pd.read_msgpack(i) for i in level3_final_paths], sort=True)

In [19]:
level_3_final = level_3_final[['consensus_cluster_rescued_anno', 'umap_0', 'umap_1', 'tsne_0', 'tsne_1']]
level_3_final.columns = ['SubType', 'l3-umap_0', 'l3-umap_1', 'l3-tsne_0', 'l3-tsne_1']
level_3_final['SubType'] = level_2_final['MajorType'] + ' ' + level_3_final['SubType']
level_3_final.loc[level_2_final['MajorType'] == 'Outlier', 'SubType'] = 'Outlier'

level_3_final.head()

,SubType,l3-umap_0,l3-umap_1,l3-tsne_0,l3-tsne_1
10E_M_1625,ASC cortex-olf,-1.541968,2.140316,-7.792882,30.459164
10E_M_1782,ASC str-hpf,12.281380,3.868788,5.614752,-35.372500
10E_M_1816,ASC str-hpf,6.088314,-0.233106,-11.659219,-5.471584
10E_M_1996,ASC mid,3.199804,2.194613,-6.513051,9.146799
10E_M_2000,ASC str-hpf,12.986892,2.863289,1.995223,-38.551920


In [20]:
for major, sub_df in level_3_final.groupby(level_2_final['MajorType']):
    print(major)
    for sub_cluster, sub_cluster_df in sub_df.groupby('SubType'):
        print(f'{sub_cluster_df.shape[0]:8d}  {sub_cluster}')

ANP
     121  ANP anp-dg
     210  ANP anp-olf-cnu
ASC
     406  ASC cortex-olf
     734  ASC mid
    1075  ASC str-hpf
CA1
     419  CA1 Ak5
    2765  CA1 Chrm3
     133  CA1 Kif26a
      54  CA1 Lingo2
     136  CA1 Outlier
     568  CA1 Ptprg
CA3
    2284  CA3 Cadm2
     139  CA3 Efnb2
CA3-St18
     107  CA3-St18 Epha5
      81  CA3-St18 Nuak1
     220  CA3-St18 Tead1
CGE-Lamp5
     243  CGE-Lamp5 Dock5
     237  CGE-Lamp5 Grid1
     732  CGE-Lamp5 Grk5
      83  CGE-Lamp5 Nrxn3
     154  CGE-Lamp5 Sorcs1
CGE-Vip
     180  CGE-Vip Ccser1
     181  CGE-Vip Clstn2
     234  CGE-Vip Fstl4
     379  CGE-Vip Galnt17
     103  CGE-Vip Grm8
      65  CGE-Vip Ntng1
     421  CGE-Vip Ptprm
     183  CGE-Vip Robo1
CLA
     118  CLA Bcl11a
     505  CLA Cdh8
     322  CLA Nrp2
CT-L6
     664  CT-L6 Hcrtr2
    4361  CT-L6 Il1rap
     869  CT-L6 Map4
    2822  CT-L6 Megf9
Chd7
      79  Chd7 Kcnc2
     153  Chd7 Megf11
     154  Chd7 Trpc7
D1L-Fstl4
     197  D1L-Fstl4 Cadm1
     386  D1L-Fstl4 

In [21]:
print('Level 3 Non Outlier Cells:', (level_3_final['SubType'].apply(lambda i: 'Outlier' not in i)).sum())

Level 3 Non Outlier Cells: 103568


## Combine

In [22]:
total_cell_meta_with_na = pd.concat([input_cell_meta, level_1_final, level_2_final, level_3_final], axis=1)

# minimum filter for consistancy, since early region do not have <500000 cells
total_cell_meta_with_na = total_cell_meta_with_na[total_cell_meta_with_na['FinalReads'] > 500000].copy()
total_cell_meta_with_na.shape

(110294, 37)

In [23]:
total_cell_meta_with_na['CellClass'] = total_cell_meta_with_na['CellClass'].fillna('Outlier')

## Re-define cell class into three

In [24]:
def cell_class_rename(cell_class, major_type):
    if cell_class in ['Exc', 'Exc-HPF']:
        return 'Exc'
    elif cell_class in ['Inh', 'NonN', 'Outlier']:
        return cell_class
    elif cell_class == 'GC':
        if major_type == 'DG':
            return 'Exc'  # DG granule join exc in dendrogram
        elif major_type == 'OLF':
            return 'Inh'  # OLF granule is Inh
        elif major_type == 'ANP':
            return 'NonN'  # Non-Mature-Neuron
        elif major_type == 'Outlier':
            return 'Outlier'
        else:
            raise ValueError(cell_class, major_type)
    else:
        raise ValueError(cell_class, major_type)


total_cell_meta_with_na['L1CellClass'] = total_cell_meta_with_na['CellClass'].copy()
total_cell_meta_with_na['CellClass'] = total_cell_meta_with_na[[
    'CellClass', 'MajorType'
]].apply(lambda i: cell_class_rename(*i.tolist()), axis=1)

In [25]:
total_cell_meta_with_na['CellClass'].value_counts() / total_cell_meta_with_na.shape[0]

Exc        0.611837
Inh        0.256995
NonN       0.074809
Outlier    0.056358
Name: CellClass, dtype: float64

## Tri-class UMAP TSNE

In [26]:
tri_class_final_paths = list(pathlib.Path('/home/hanliu/project/mouse_rostral_brain/').glob('study/ThreeCellClass/*/Coords/cell_tidy_data.msg'))
tri_class_final = pd.concat([pd.read_msgpack(i) for i in tri_class_final_paths], sort=True)
tri_class_coords = tri_class_final[['tsne_0', 'tsne_1', 'umap_0', 'umap_1']]
tri_class_coords.columns = ['class_tsne_0', 'class_tsne_1', 'class_umap_0', 'class_umap_1']
tri_class_coords = tri_class_coords.reindex(total_cell_meta_with_na.index).fillna(0)
total_cell_meta_with_na = pd.concat([total_cell_meta_with_na, tri_class_coords], axis=1)

In [27]:
print(tri_class_final.shape[0], 'cells has tri-class coords')

103982 cells has tri-class coords


## Update region info

In [28]:
region_info = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/BrainRegion.txt', 
                          sep='\t', index_col='DissectionRegion').fillna('')

In [29]:
expend_region_info = total_cell_meta_with_na['Region'].apply(lambda i: region_info.loc[i])
for col_name, col in expend_region_info.iteritems():
    total_cell_meta_with_na[col_name] = col

In [30]:
# add replicate column
total_cell_meta_with_na['Replicate'] = total_cell_meta_with_na[
    'Region'] + '-' + total_cell_meta_with_na['FACS_Date'].astype(str)

## Save file

In [31]:
total_cell_meta_with_na.isna().sum()

AllcPath                     0
CCC_Rate                     0
CG_Rate                      0
CG_RateAdj                   0
CH_Rate                      0
CH_RateAdj                   0
FinalReads                   0
InputReads                   0
MappedReads                  0
Region                       0
index_name                   0
uid                          0
BamFilteringRate             0
MappingRate                  0
Pos96                        0
Plate                        0
Col96                        0
Row96                        0
Col384                       0
Row384                       0
FACS_Date                    0
Slice                        0
CellClass                    0
l1-umap_0                 5954
l1-umap_1                 5954
l1-tsne_0                 5954
l1-tsne_1                 5954
MajorType                 5954
l2-umap_0                 5954
l2-umap_1                 5954
l2-tsne_0                 5954
l2-tsne_1                 5954
SubType 

In [32]:
total_cell_meta_with_na.to_msgpack('TotalClusteringResults.withOutlierNA.msg')

In [33]:
total_cell_meta_no_na = total_cell_meta_with_na.dropna(subset=['CellClass', 'MajorType', 'SubType'], how='any')

In [34]:
assert total_cell_meta_no_na.isna().sum().sum() == 0
total_cell_meta_no_na.to_msgpack('TotalClusteringResults.msg')

In [35]:
total_cell_meta_no_na.head()

,AllcPath,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,Region,...,MajorRegion,SubRegion,DetailRegion,PotentialOverlap (MMB),Anterior (CCF coords),Posterior (CCF coords),MajorRegionColor,SubRegionColor,DissectionRegionColor,Replicate
10E_M_0,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.008198,0.822633,0.821166,0.041640,0.033718,1626504.0,4407752,2892347.0,10E,...,HPF,CA1-3,"CA1, CA2, CA3, SUB, ProS","PA, HATA",7500,8100,#d62728,#d62728,#FF6347,10E-190625
10E_M_1,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.006019,0.743035,0.741479,0.024127,0.018218,2009998.0,5524084,3657352.0,10E,...,HPF,CA1-3,"CA1, CA2, CA3, SUB, ProS","PA, HATA",7500,8100,#d62728,#d62728,#FF6347,10E-190625
10E_M_10,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.006569,0.750172,0.748520,0.027665,0.021235,1383636.0,3455260,2172987.0,10E,...,HPF,CA1-3,"CA1, CA2, CA3, SUB, ProS","PA, HATA",7500,8100,#d62728,#d62728,#FF6347,10E-190625
10E_M_101,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.006353,0.760898,0.759369,0.026547,0.020323,2474670.0,7245482,4778768.0,10E,...,HPF,CA1-3,"CA1, CA2, CA3, SUB, ProS","PA, HATA",7500,8100,#d62728,#d62728,#FF6347,10E-190625
10E_M_102,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.005409,0.752980,0.751637,0.019497,0.014164,2430290.0,7004754,4609570.0,10E,...,HPF,CA1-3,"CA1, CA2, CA3, SUB, ProS","PA, HATA",7500,8100,#d62728,#d62728,#FF6347,10E-190625


In [36]:
print(total_cell_meta_with_na.shape[0], 'cells including outlier')
print(total_cell_meta_no_na.shape[0], 'cells excluding outlier')

110294 cells including outlier
103982 cells excluding outlier


In [37]:
total_cell_meta_no_na['CellClass'].value_counts()

Exc     67472
Inh     28343
NonN     8167
Name: CellClass, dtype: int64

## Plot

In [38]:
cell_tidy_data = total_cell_meta_no_na
coord_base='l1-umap'

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(20, 10), dpi=300, ncols=2)
categorical_scatter(data=cell_tidy_data,
                    coord_base=coord_base,
                    ax=ax1,
                    scatter_kws=dict(s=0.7),
                    text_anno_kws=dict(fontsize=5),
                    max_points=None,
                    hue='CellClass',
                    palette='tab20',
                    text_anno='CellClass')
ax1.set(title='Level 1 - Cell Class')

categorical_scatter(data=cell_tidy_data,
                    coord_base=coord_base,
                    ax=ax2,
                    scatter_kws=dict(s=0.7),
                    text_anno_kws=dict(fontsize=5),
                    max_points=None,
                    hue='MajorType',
                    palette='tab20',
                    text_anno='MajorType')
ax2.set(title='Level 2 - MajorType')

fig.suptitle('Cluster Annotation')

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(20, 10), dpi=300, ncols=2)
categorical_scatter(data=cell_tidy_data,
                    coord_base=coord_base,
                    ax=ax1,
                    scatter_kws=dict(s=0.7),
                    text_anno_kws=dict(fontsize=5),
                    max_points=None,
                    hue='MajorRegion',
                    palette=major_region_palette,
                    text_anno='MajorType')
ax1.set(title='Major Region')

categorical_scatter(data=cell_tidy_data,
                    coord_base=coord_base,
                    ax=ax2,
                    scatter_kws=dict(s=0.7),
                    text_anno_kws=dict(fontsize=5),
                    max_points=None,
                    hue='SubRegion',
                    palette=sub_region_palette,
                    text_anno='MajorType')
ax2.set(title='Sub-Region')

fig.suptitle('Brain Region Annotation')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=300)
categorical_scatter(data=cell_tidy_data,
                    coord_base=coord_base,
                    ax=ax,
                    scatter_kws=dict(s=0.7),
                    text_anno_kws=dict(fontsize=5),
                    max_points=None,
                    show_legend=True,
                    legend_kws=dict(ncol=2, bbox_to_anchor=(1.05, 0.9)),
                    hue='SubRegion',
                    palette=sub_region_palette,
                    text_anno='MajorType'
                   )
ax.set(title='Sub-Region')

In [39]:
# sub_type_google_sheet = total_cell_meta_no_na.groupby(['CellClass', 'MajorType', 'SubType']).apply(lambda i: i.shape[0]).reset_index()
# sub_type_google_sheet.columns = ['CellClass', 'MajorType', 'SubType', 'CellCount']
# sub_type_google_sheet['Markers'] = sub_type_google_sheet['SubType'].apply(lambda i: i.split(' ')[1])
# sub_type_google_sheet = sub_type_google_sheet[sub_type_google_sheet['Markers'] != 'Outlier']
# print(sub_type_google_sheet.to_csv(sep='\t', index=None))